In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from mpmath import besseljzero
from scipy.special import gamma
from scipy.special import jv, iv, ive
from scipy.optimize import minimize
from scipy.optimize import differential_evolution
from scipy.interpolate import interp1d

from scipy.stats import pearsonr
from sklearn.metrics import r2_score

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def simulate_HSDM_4D(a, mu, ndt, sigma=1, dt=0.001):
    x = np.zeros(mu.shape)
    
    rt = 0
    
    while np.linalg.norm(x, 2) < a(rt):
        x += mu*dt + sigma*np.sqrt(dt)*np.random.normal(0, 1, mu.shape)
        rt += dt
    
    theta1 = np.arctan2(np.sqrt(x[3]**2 + x[2]**2 + x[1]**2), x[0])
    theta2 = np.arctan2(np.sqrt(x[3]**2 + x[2]**2), x[1])
    theta3 = np.arctan2(x[3], x[2])
    
    
    return rt+ndt, (theta1, theta2, theta3)

In [3]:
def k(a, da, t, q, sigma=2):
    return 0.5 * (q - 0.5*sigma - da(t))

def psi(a, da, t, z, tau, q, sigma=2):
    kk = k(a, da, t, q, sigma)
    
    if 2*np.sqrt(a(t)*z)/(sigma*(t-tau))<=700:
        term1 = 1./(sigma*(t - tau)) * np.exp(- (a(t) + z)/(sigma*(t-tau)))
        term2 = (a(t)/z)**(0.5*(q-sigma)/sigma)
        term3 = da(t) - (a(t)/(t-tau)) + kk
        term4 = iv(q/sigma-1, 2*np.sqrt(a(t)*z)/(sigma*(t-tau)))
        term5 = (np.sqrt(a(t)*z)/(t-tau)) * iv(q/sigma, 2*np.sqrt(a(t)*z)/(sigma*(t-tau)))
    else:
        term1 = 1./(sigma*(t - tau))
        term2 = (a(t)/z)**(0.5*(q-sigma)/sigma)
        term3 = da(t) - (a(t)/(t-tau)) + kk
        term4 = ive(q/sigma-1, (a(t) + z)/(sigma*(t-tau)))
        term5 = (np.sqrt(a(t)*z)/(t-tau)) * ive(q/sigma, (a(t) + z)/(sigma*(t-tau)))
    
    return term1 * term2 * (term3 * term4 + term5)

def ie_bessel_fpt(a, da, q, z, sigma=2, dt=0.1, T_max=2):
    g = [0]
    T = [0]
    g.append(-2*psi(a, da, dt, z, 0, q, sigma))
    T.append(dt)
    
    for n in range(2, int(T_max/dt)+2):
        s = -2 * psi(a, da, n*dt, z, 0, q, sigma)

        for j in range(1, n):
            if a(j*dt) == 0:
                continue
            
            s += 2 * dt * g[j] * psi(a, da, n*dt, a(j*dt), j*dt, q, sigma)

        g.append(s)
        T.append(n*dt)
        
    g = np.asarray(g)
    T = np.asarray(T)
    
    gt = interp1d(T, g)
    return gt

In [4]:
def HSDM_4D_likelihood(prms, RT, Theta):
    a = lambda t: prms[0] - prms[1]*t
    a2 = lambda t: (a(t))**2
    da2 = lambda t: -2*prms[1] * a(t)
    
    ndt = prms[2]
    mu = np.array([prms[3], prms[4], prms[5], prms[6]])
    
    T_max = min(max(RT), prms[0]/prms[1])
    fpt = ie_bessel_fpt(a2, da2, mu.shape[0], 0.000001, 
                        dt=0.02, T_max=T_max)
    
    log_lik = 0
    for i in range(len(RT)):
        rt, theta = RT[i], Theta[i]
        if rt - ndt > 0.0001 and rt - ndt < T_max:
            mu_dot_x0 = mu[0]*np.cos(theta[0])
            mu_dot_x1 = mu[1]*np.sin(theta[0])*np.cos(theta[1]) 
            mu_dot_x2 = mu[2]*np.sin(theta[0])*np.sin(theta[1])*np.cos(theta[2])
            mu_dot_x3 = mu[3]*np.sin(theta[0])*np.sin(theta[1])*np.sin(theta[2]) 
            term1 = a(rt - ndt) * (mu_dot_x0 + mu_dot_x1 + mu_dot_x2 + mu_dot_x3)
            term2 = 0.5 * np.linalg.norm(mu, 2)**2 * (rt-ndt)
            
            density = np.exp(term1 - term2) * fpt(rt - ndt)
            
            if 0.1**14 < density:
                log_lik += -np.log(density)
            else:
                log_lik += -np.log(0.1**14)
        else:
            log_lik += -np.log(0.1**14)
        
    return log_lik

In [5]:
recovery_df = {'b0_true': [],
               'b0_estimate': [],
               'lambda_true': [],
               'lambda_estimate': [],
               'ndt_true': [],
               'ndt_estimate': [],
               'mu1_true': [],
               'mu1_estimate': [],
               'mu2_true': [],
               'mu2_estimate': [],
               'mu3_true': [],
               'mu3_estimate': [],
               'mu4_true': [],
               'mu4_estimate': []}

min_b0 = 2
max_b0 = 5

min_lambda = .1
max_lambda = 2

min_ndt = 0.1
max_ndt = 1

min_mu = -3
max_mu = 3

In [6]:
for n in tqdm(range(50)):
    b0 = np.random.uniform(min_b0, max_b0)
    lamb = np.random.uniform(min_lambda, max_lambda)
    a = lambda t: b0 - lamb*t
    
    ndt = np.random.uniform(min_ndt, max_ndt)
    mu = np.array([np.random.uniform(min_mu, max_mu), 
                   np.random.uniform(min_mu, max_mu),
                   np.random.uniform(min_mu, max_mu),
                   np.random.uniform(min_mu, max_mu)])
    
    RT = []
    Theta = []
    
    for i in range(750):
        rt, theta = simulate_HSDM_4D(a, mu, ndt)
        RT.append(rt)
        Theta.append(theta)
    
    min_ans = differential_evolution(HSDM_4D_likelihood,
                                     args=(RT, Theta), 
                                     bounds=[(min_b0, max_b0), (min_lambda, max_lambda),
                                             (min_ndt, max_ndt), (min_mu, max_mu), 
                                             (min_mu, max_mu), (min_mu, max_mu), (min_mu, max_mu)])
    
    min_ans = minimize(HSDM_4D_likelihood,
                       args=(RT, Theta),
                       method='nelder-mead',
                       x0=min_ans.x,
                       bounds=[(min_b0, max_b0), (min_lambda, max_lambda), (min_ndt, max_ndt),
                                (min_mu, max_mu), (min_mu, max_mu), (min_mu, max_mu), (min_mu, max_mu)])    
    
    if min_ans.success:
        recovery_df['b0_true'].append(b0)
        recovery_df['lambda_true'].append(lamb)
        recovery_df['ndt_true'].append(ndt)
        recovery_df['mu1_true'].append(mu[0])
        recovery_df['mu2_true'].append(mu[1])
        recovery_df['mu3_true'].append(mu[2])
        recovery_df['mu4_true'].append(mu[3])

        recovery_df['b0_estimate'].append(min_ans.x[0])
        recovery_df['lambda_estimate'].append(min_ans.x[1])
        recovery_df['ndt_estimate'].append(min_ans.x[2])
        recovery_df['mu1_estimate'].append(min_ans.x[3])
        recovery_df['mu2_estimate'].append(min_ans.x[4])
        recovery_df['mu3_estimate'].append(min_ans.x[5])
        recovery_df['mu4_estimate'].append(min_ans.x[6])

100%|██████████████████████████████| 50/50 [3:18:14<00:00, 237.90s/it]


In [7]:
recovery_df = pd.DataFrame(recovery_df)
recovery_df.corr()

,b0_true,b0_estimate,lambda_true,lambda_estimate,ndt_true,ndt_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate,mu3_true,mu3_estimate,mu4_true,mu4_estimate
b0_true,1.000000,0.885074,-0.137768,-0.207539,-0.173618,-0.115634,0.228611,0.225221,-0.007946,-0.002396,-0.269686,-0.274993,0.252384,0.257846
b0_estimate,0.885074,1.000000,-0.293738,-0.089559,-0.135414,-0.184331,0.333060,0.328365,0.053723,0.062073,-0.232755,-0.242472,0.233288,0.246141
lambda_true,-0.137768,-0.293738,1.000000,0.814117,0.097025,0.169397,-0.021441,-0.025091,-0.121806,-0.133211,-0.019360,-0.014871,-0.329665,-0.337026
lambda_estimate,-0.207539,-0.089559,0.814117,1.000000,0.145160,0.099828,0.115024,0.109448,-0.041187,-0.048335,-0.039110,-0.042414,-0.339645,-0.336757
ndt_true,-0.173618,-0.135414,0.097025,0.145160,1.000000,0.972807,-0.183888,-0.182999,0.233551,0.233745,-0.161344,-0.146787,-0.265118,-0.257605
ndt_estimate,-0.115634,-0.184331,0.169397,0.099828,0.972807,1.000000,-0.226034,-0.224776,0.201003,0.200215,-0.189737,-0.173771,-0.255962,-0.251765
mu1_true,0.228611,0.333060,-0.021441,0.115024,-0.183888,-0.226034,1.000000,0.999377,-0.120574,-0.118062,0.040143,0.028982,-0.007073,-0.006196
mu1_estimate,0.225221,0.328365,-0.025091,0.109448,-0.182999,-0.224776,0.999377,1.000000,-0.118410,-0.116131,0.036164,0.025131,-0.015052,-0.014323
mu2_true,-0.007946,0.053723,-0.121806,-0.041187,0.233551,0.201003,-0.120574,-0.118410,1.000000,0.998759,-0.000839,0.001527,-0.066467,-0.060126
mu2_estimate,-0.002396,0.062073,-0.133211,-0.048335,0.233745,0.200215,-0.118062,-0.116131,0.998759,1.000000,-0.001297,0.000316,-0.061339,-0.055082


In [8]:
file_name = '_Recovery_data/IE_4d_recovery_linear_750_02.csv'
old_recovery_data = pd.read_csv(file_name, index_col=0)
recovery_df = pd.concat([old_recovery_data, 
                         recovery_df]).reset_index(drop=True)
recovery_df.to_csv(file_name)

In [9]:
recovery_df

,b0_true,b0_estimate,lambda_true,lambda_estimate,ndt_true,ndt_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate,mu3_true,mu3_estimate,mu4_true,mu4_estimate
0,2.148569,2.127918,1.973173,1.969177,0.681601,0.692225,2.500912,2.568808,2.337256,2.548624,2.331869,2.343101,-1.909468,-1.981261
1,3.329892,4.080411,0.747561,1.299172,0.312853,0.211161,-1.931150,-1.753433,-2.307423,-2.155293,1.979357,1.903506,-0.270857,-0.210224
2,2.708208,2.889251,1.490681,1.695348,0.321848,0.301423,2.531152,2.484677,-1.218835,-1.123547,0.202079,0.219951,0.164059,0.163304
3,2.104836,2.017831,0.250207,0.100249,0.539044,0.553434,-1.576189,-1.645340,-0.602918,-0.570927,-2.877802,-2.850193,-0.632633,-0.632674
4,3.038651,2.867793,1.773815,1.590632,0.872998,0.903561,-0.564066,-0.516428,2.131152,2.289284,0.181543,0.186885,0.597900,0.603951
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,4.026612,4.297430,0.134702,0.240004,0.899746,0.851239,-0.130033,-0.074419,1.011337,0.938627,2.517350,2.501800,2.402357,2.405700
296,2.054020,2.000000,0.926530,0.864207,0.400096,0.403141,-2.471919,-2.363079,-0.747728,-0.748174,0.155650,0.136487,1.573555,1.556417
297,2.657576,3.107445,0.627873,0.995051,0.667820,0.590134,-0.849494,-0.791809,1.769818,1.652426,0.361283,0.389053,-1.096540,-0.997944
298,2.166715,2.088800,0.926905,0.896164,0.938847,0.959296,-0.913265,-0.907674,-0.514649,-0.445925,0.551926,0.633877,-2.242899,-2.253194
